In [1]:
import heapq
from typing import Dict, List, Tuple, Set
from dataclasses import dataclass
import time

# Definição do grafo da Romênia com distâncias reais entre cidades
GRAFO_ROMANIA = {
    'Arad': [('Zerind', 75), ('Timisoara', 118), ('Sibiu', 140)],
    'Zerind': [('Arad', 75), ('Oradea', 71)],
    'Oradea': [('Zerind', 71), ('Sibiu', 151)],
    'Timisoara': [('Arad', 118), ('Lugoj', 111)],
    'Lugoj': [('Timisoara', 111), ('Mehadia', 70)],
    'Mehadia': [('Lugoj', 70), ('Drobeta', 75)],
    'Drobeta': [('Mehadia', 75), ('Craiova', 120)],
    'Craiova': [('Drobeta', 120), ('Rimnicu Vilcea', 146), ('Pitesti', 138)],
    'Sibiu': [('Arad', 140), ('Oradea', 151), ('Fagaras', 99), ('Rimnicu Vilcea', 80)],
    'Rimnicu Vilcea': [('Sibiu', 80), ('Craiova', 146), ('Pitesti', 97)],
    'Fagaras': [('Sibiu', 99), ('Bucharest', 211)],
    'Pitesti': [('Rimnicu Vilcea', 97), ('Craiova', 138), ('Bucharest', 101)],
    'Bucharest': [('Fagaras', 211), ('Pitesti', 101), ('Giurgiu', 90), ('Urziceni', 85)],
    'Giurgiu': [('Bucharest', 90)],
    'Urziceni': [('Bucharest', 85), ('Hirsova', 98), ('Vaslui', 142)],
    'Hirsova': [('Urziceni', 98), ('Eforie', 86)],
    'Eforie': [('Hirsova', 86)],
    'Vaslui': [('Urziceni', 142), ('Iasi', 92)],
    'Iasi': [('Vaslui', 92), ('Neamt', 87)],
    'Neamt': [('Iasi', 87)]
}

# Heurística: Distância em linha reta até Bucareste
HEURISTICA_BUCARESTE = {
    'Arad': 366,
    'Bucharest': 0,
    'Craiova': 160,
    'Drobeta': 242,
    'Eforie': 161,
    'Fagaras': 176,
    'Giurgiu': 77,
    'Hirsova': 151,
    'Iasi': 226,
    'Lugoj': 244,
    'Mehadia': 241,
    'Neamt': 234,
    'Oradea': 380,
    'Pitesti': 100,
    'Rimnicu Vilcea': 193,
    'Sibiu': 253,
    'Timisoara': 329,
    'Urziceni': 80,
    'Vaslui': 199,
    'Zerind': 374
}


In [2]:
@dataclass
class No:
    """Representa um nó na busca"""
    cidade: str
    caminho: List[str]
    custo_total: int
    
    def __lt__(self, outro):
        # Para o heapq comparar, usamos apenas a heurística (busca gulosa!)
        return HEURISTICA_BUCARESTE[self.cidade] < HEURISTICA_BUCARESTE[outro.cidade]



In [ ]:
def busca_gulosa(inicio: str, objetivo: str, verbose: bool = True) -> Tuple[List[str], int, Dict]:
    """
    Implementa a Busca Gulosa (Greedy Best-First Search)
    
    Args:
        inicio: Cidade inicial
        objetivo: Cidade objetivo
        verbose: Se True, imprime o processo de busca
        
    Returns:
        Tupla com (caminho, custo_total, estatisticas)
    """
    
    # Estatísticas
    stats = {
        'nos_expandidos': 0,
        'nos_visitados': 0,
        'tempo_execucao': 0
    }
    
    inicio_tempo = time.time()
    
    # Fila de prioridade (heap) - ordenada pela heurística
    fronteira = []
    heapq.heappush(fronteira, No(inicio, [inicio], 0))
    
    # Conjunto de cidades já visitadas
    visitados: Set[str] = set()
    
    if verbose:
        print("=" * 70)
        print("🚀 BUSCA GULOSA: DE ARAD ATÉ BUCARESTE")
        print("=" * 70)
        print(f"\n📍 Início: {inicio}")
        print(f"🎯 Objetivo: {objetivo}")
        print(f"📏 Heurística inicial: {HEURISTICA_BUCARESTE[inicio]} km\n")
        print("-" * 70)
    
    while fronteira:
        # Pega o nó com MENOR heurística (mais promissor)
        no_atual = heapq.heappop(fronteira)
        stats['nos_expandidos'] += 1
        
        if verbose:
            print(f"\n🔍 Expandindo: {no_atual.cidade}")
            print(f"   Caminho até aqui: {' → '.join(no_atual.caminho)}")
            print(f"   Custo real acumulado: {no_atual.custo_total} km")
            print(f"   Heurística (dist. linha reta): {HEURISTICA_BUCARESTE[no_atual.cidade]} km")
        
        # Chegamos no objetivo!
        if no_atual.cidade == objetivo:
            stats['tempo_execucao'] = time.time() - inicio_tempo
            if verbose:
                print("\n" + "=" * 70)
                print("🎉 OBJETIVO ALCANÇADO!")
                print("=" * 70)
            return no_atual.caminho, no_atual.custo_total, stats
        
        # Marca como visitado
        if no_atual.cidade in visitados:
            continue
        visitados.add(no_atual.cidade)
        stats['nos_visitados'] += 1
        
        # Expande os vizinhos
        if verbose:
            print(f"   Vizinhos de {no_atual.cidade}:")
        
        for vizinho, custo in GRAFO_ROMANIA[no_atual.cidade]:
            if vizinho not in visitados:
                novo_caminho = no_atual.caminho + [vizinho]
                novo_custo = no_atual.custo_total + custo
                novo_no = No(vizinho, novo_caminho, novo_custo)
                heapq.heappush(fronteira, novo_no)
                
                if verbose:
                    print(f"      → {vizinho}: custo +{custo}km, h={HEURISTICA_BUCARESTE[vizinho]}km")
    
    # Não encontrou solução
    stats['tempo_execucao'] = time.time() - inicio_tempo
    return [], 0, stats


In [ ]:
def imprimir_resultado(caminho: List[str], custo: int, stats: Dict):
    """Imprime o resultado formatado da busca"""
    print("\n" + "=" * 70)
    print("📊 RESULTADO FINAL")
    print("=" * 70)
    
    if caminho:
        print(f"\n✅ Caminho encontrado:")
        print(f"   {' → '.join(caminho)}")
        print(f"\n💰 Custo total: {custo} km")
        print(f"🔢 Número de cidades: {len(caminho)}")
        print(f"\n📈 Estatísticas:")
        print(f"   • Nós expandidos: {stats['nos_expandidos']}")
        print(f"   • Nós visitados: {stats['nos_visitados']}")
        print(f"   • Tempo de execução: {stats['tempo_execucao']:.6f} segundos")
        
        # Mostra o custo de cada trecho
        print(f"\n🗺️  Detalhamento do percurso:")
        custo_acumulado = 0
        for i in range(len(caminho) - 1):
            cidade_atual = caminho[i]
            proxima_cidade = caminho[i + 1]
            
            # Encontra o custo entre as cidades
            for vizinho, custo_trecho in GRAFO_ROMANIA[cidade_atual]:
                if vizinho == proxima_cidade:
                    custo_acumulado += custo_trecho
                    print(f"   {i+1}. {cidade_atual} → {proxima_cidade}: {custo_trecho} km (acumulado: {custo_acumulado} km)")
                    break
    else:
        print("\n❌ Nenhum caminho encontrado!")
    
    print("\n" + "=" * 70)

In [ ]:
def comparar_com_caminho_otimo():
    """Compara o resultado da busca gulosa com o caminho ótimo conhecido"""
    print("\n" + "🔬 ANÁLISE COMPARATIVA")
    print("=" * 70)
    
    # Caminho ótimo conhecido (encontrado por A*)
    caminho_otimo = ['Arad', 'Sibiu', 'Rimnicu Vilcea', 'Pitesti', 'Bucharest']
    custo_otimo = 418  # 140 + 80 + 97 + 101
    
    print(f"\n🏆 Caminho ÓTIMO (A*):")
    print(f"   {' → '.join(caminho_otimo)}")
    print(f"   Custo: {custo_otimo} km")
    
    # Executa busca gulosa sem verbose para comparação
    caminho_gulosa, custo_gulosa, _ = busca_gulosa('Arad', 'Bucharest', verbose=False)
    
    print(f"\n🎯 Caminho encontrado pela BUSCA GULOSA:")
    print(f"   {' → '.join(caminho_gulosa)}")
    print(f"   Custo: {custo_gulosa} km")
    
    diferenca = custo_gulosa - custo_otimo
    percentual = (diferenca / custo_otimo) * 100
    
    print(f"\n📊 Diferença:")
    print(f"   • Distância extra: {diferenca} km")
    print(f"   • Percentual: {percentual:.2f}% mais longo que o ótimo")
    
    if caminho_gulosa == caminho_otimo:
        print(f"\n✨ Uau! A busca gulosa encontrou o caminho ótimo! 🎉")
    else:
        print(f"\n💡 A busca gulosa é rápida, mas não garantiu o caminho ótimo.")
        print(f"   Para garantir o ótimo, use A* (considera custo + heurística)!")



In [ ]:
def main():
    """Função principal"""
    print("\n🇷🇴 PROBLEMA DA ROMÊNIA - BUSCA GULOSA 🇷🇴\n")
    
    # Executa a busca
    caminho, custo, stats = busca_gulosa('Arad', 'Bucharest', verbose=True)
    
    # Imprime o resultado
    imprimir_resultado(caminho, custo, stats)
    
    # Comparação com o ótimo
    comparar_com_caminho_otimo()
    
    print("\n💡 LIÇÃO: A Busca Gulosa é gulosa mesmo! Ela vai sempre na direção")
    print("   que PARECE melhor (menor heurística), mas pode não encontrar o")
    print("   caminho mais curto. É rápida, mas não garante otimalidade! 🚀\n")


if __name__ == "__main__":
    main()